In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Convolutional Network V1

In [6]:
# Constants
TEST_DATA_DIR = 'data/test'
TRAIN_DATA_DIR = 'data/train'
TRAIN_DATA_LABELS = 'data/train.csv'

IMG_WIDTH = 32
IMG_HEIGTH = 32
BATCH_SIZE = 128
EPOCHS = 30

In [8]:
# Data Generator tutorial: https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGTH),
    batch_size=BATCH_SIZE, class_mode='binary')


Found 17500 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
